### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)


#Create dataframe of data grouped by Screen Name - used often later
groupedBySN = purchase_data.groupby("SN").last()

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [42]:
#Create function to apply formatting
def currFormat(x):
        return "${:.2f}".format(x)

## Player Count

* Display the total number of players


In [43]:
totalPlayers = purchase_data["SN"].nunique()
totalPlayersDF = pd.DataFrame({"Total Players" : totalPlayers}, index=[0])
totalPlayersDF

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [44]:
numUniqueItems = purchase_data["Item Name"].nunique() #Get number of unique items
avgPrice = purchase_data["Price"].mean() #Get average item price
numPurchases = purchase_data["Purchase ID"].count() #Get total number of purchases
totalRev = purchase_data["Price"].sum() #Get total revenue from purchases


purchaseSummaryDF = pd.DataFrame({"Number of Unique Items" : numUniqueItems, 
                                    "Average Price" : f'${round(avgPrice, 2)}', #format average price
                                    "Number of Purchases" : numPurchases, 
                                    "Total Revenue" : f'${round(totalRev,2)}'}, #format total rev
                                    index = [0])
purchaseSummaryDF

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [45]:
#Get count of players of each gender and place back in dataframe
genderCounts = groupedBySN.groupby("Gender")["Gender"].count()
genderCountsDF = pd.DataFrame(genderCounts).rename(columns={"Gender" : "Total Count"})

#Calculate % Column
genderCountsDF["Percentage of Players"] = (100 * genderCountsDF["Total Count"] / totalPlayers)
genderCountsDF["Percentage of Players"] = genderCountsDF["Percentage of Players"].round(2) #Round Result
genderCountsDF["Percentage of Players"] = genderCountsDF["Percentage of Players"].astype(str) + "%" #Add percent sign

#Display Data
genderCountsDF

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [46]:
#Get purchase count, average purchase, total purchases
summaryData = purchase_data.groupby("Gender").aggregate({"Gender" : "count", "Price" : ["mean", "sum"]})

#Get rid of outer column hierarchy
summaryData.columns = summaryData.columns.droplevel(0)

#Rename columns
summaryData = summaryData.rename(columns={"count" : "Purchase Count", 
                                          "mean" : "Average Purchase Price", 
                                          "sum" : "Total Purchase Value"})

#Get average total purchase by person for each gender
#Get each player's total purchases but retain their gender
tempDF = purchase_data.groupby(["SN", "Gender"])["Price"].sum().reset_index()

#Get average total purchase per person for each gender
tempDF = tempDF.groupby("Gender").mean()

#Add to summary table and rename new column
summaryData = summaryData.join(tempDF)
summaryData = summaryData.rename({"Price" : "Avg Total Purchase per Person"}, axis=1)

#Clean up formatting
summaryData["Average Purchase Price"] = summaryData["Average Purchase Price"].round(2).apply(currFormat)
summaryData["Total Purchase Value"] = summaryData["Total Purchase Value"].round(2).apply(currFormat)
summaryData["Avg Total Purchase per Person"] = summaryData["Avg Total Purchase per Person"].round(2).apply(currFormat)

summaryData

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [47]:
#Setup bin ranges and labels
bin_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
bins = [0, 9, 14, 19, 24, 29, 34, 39, 150]

#New approach
#Get list of bins for each SN and replace in dataframe
tempDF = pd.cut(groupedBySN["Age"], bins, labels=bin_labels, right=True)
tempDF = pd.DataFrame(tempDF)

#Get count of each bin, then sort the bins in order
ageBinned = pd.DataFrame(tempDF["Age"].value_counts()).rename(columns={"Age" : "Total Count"})
ageBinned = ageBinned.sort_index()

#Calculate percentage of playerbase, then round to 2 decimal places
ageBinned["Percentage of Players"] = (100 * ageBinned["Total Count"] / totalPlayers).round(2)

#Add % Symbol
ageBinned["Percentage of Players"] = ageBinned["Percentage of Players"].astype(str) + "%"

ageBinned

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [48]:
#Setup bin ranges and labels
bin_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
bins = [0, 9, 14, 19, 24, 29, 34, 39, 150]

#New approach
#Get list of bins for each SN and replace in dataframe
ageBins = pd.cut(purchase_data["Age"], bins, labels=bin_labels, right=True)
ageBins = pd.DataFrame(ageBins)

#Append back to original data
purchDataBins = purchase_data.copy()
purchDataBins = purchDataBins.join(ageBins, rsuffix="_bins")

#Store a pre-groupby copy
pDBTemp = purchDataBins.copy()

#Get aggregates
purchDataBins = purchDataBins.groupby("Age_bins").aggregate({"Age_bins" : "count", "Price" : ["mean", "sum"]})

#Get rid of outer column hierarchy
purchDataBins.columns = purchDataBins.columns.droplevel(0)

#Rename columns
purchDataBins = purchDataBins.rename(columns={"count" : "Purchase Count", 
                                          "mean" : "Average Purchase Price", 
                                          "sum" : "Total Purchase Value"})

#Get average total purchase by person for each age
#Get age bins for each unique player
groupedAgeBins = pDBTemp.groupby(["SN"])["SN", "Age_bins"].last()

#Get total purchase for each unique player
tempDF = pDBTemp.groupby(["SN"])["SN", "Price"].sum()

#Add back the age bins
tempDF = tempDF.join(groupedAgeBins, rsuffix="_bins")

#Get the average total purchase by age
tempDF = tempDF.groupby("Age_bins").mean()

#Add to summary table and rename new column
purchDataBins = purchDataBins.join(tempDF)
purchDataBins = purchDataBins.rename({"Price" : "Avg Total Purchase per Person"}, axis=1)


#Clean up formatting
purchDataBins["Average Purchase Price"] = purchDataBins["Average Purchase Price"].round(2).apply(currFormat)
purchDataBins["Total Purchase Value"] = purchDataBins["Total Purchase Value"].round(2).apply(currFormat)
purchDataBins["Avg Total Purchase per Person"] = purchDataBins["Avg Total Purchase per Person"].round(2).apply(currFormat)

purchDataBins

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age_bins,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [49]:
#Get aggregate data for top spenders - group dataset by Screen Name, then get count of purchases, avg purchase, total spent
topSpenderData = purchase_data.groupby("SN").aggregate({"Purchase ID" : "count", "Price" : ["mean", "sum"]}, axis=1)

#Rename columns to something meaningful. Drop the outermost column hierarchy
topSpenderData = topSpenderData.rename(columns={"count" : "Purchase Count", "mean" : "Average Purchase Price", "sum" : "Total Purchase Value"})
topSpenderData.columns = topSpenderData.columns.droplevel(0)

#Sort the values based on Total Purchase Value
topSpenderData = topSpenderData.sort_values("Total Purchase Value", ascending=False)

#Clean up formatting
topSpenderData["Average Purchase Price"] = topSpenderData["Average Purchase Price"].round(2).apply(currFormat)
topSpenderData["Total Purchase Value"] = topSpenderData["Total Purchase Value"].round(2).apply(currFormat)

#Display top 5 spenders
topSpenderData.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [50]:
#Get Item ID, Item Name, Price columns
itemDF = purchase_data[["Item ID", "Item Name", "Price"]]

#Group by Item ID and Item Name; perform calculations (count, mean, sum)
itemDF = itemDF.groupby(["Item ID", "Item Name"]).aggregate({"Item ID" : "count", "Price" : ["mean", "sum"]})

#Get rid of outer column hierarchy
itemDF.columns = itemDF.columns.droplevel(0)

#Rename Columns
itemDF = itemDF.rename(columns={"count": "Purchase Count", "mean" : "Item Price", "sum" : "Total Purchase Value"})

#Sort Values by purchase count
itemDF = itemDF.sort_values("Purchase Count", ascending=False)

#Save data to separate dataframe to format - leave original intact for additonal operations
mostPopularItems = itemDF.copy()

#Format data
mostPopularItems["Item Price"] = mostPopularItems["Item Price"].round(2).apply(currFormat)
mostPopularItems["Total Purchase Value"] = mostPopularItems["Total Purchase Value"].round(2).apply(currFormat)
 
#Display 5 most popular items
mostPopularItems.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [51]:
#Sort data from last table by Purchase Value
itemDF = itemDF.sort_values("Total Purchase Value", ascending=False)

#Clean up formatting
itemDF["Item Price"] = itemDF["Item Price"].round(2).apply(currFormat)
itemDF["Total Purchase Value"] = itemDF["Total Purchase Value"].round(2).apply(currFormat)

#Display 5 highest revenue items (not necessarily most profitable!)
itemDF.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In [62]:
#Extra: See what % of total purchases is represented by top item
#
Top5Count = 12+9+9+9+8
Top5Rev = 50.76 + 44.10 + 41.22 + 39.04 + 34.80

Top1CountPerc = (100 * 12 / numPurchases)
Top5CountPerc = (100 * Top5Count / numPurchases)
Top1RevPerc = (100 * 50.76/totalRev)
Top5RevPerc = (100 * Top5Rev/totalRev)

print(f"Top Item share of total purchase count: {Top1CountPerc.round(2)}%")
print(f"Top 5 Items share of total purchase count: {Top5CountPerc.round(2)}%")
print(f"Top Item share of Total Revenue: {Top1RevPerc.round(2)}%")
print(f"Top 5 Items share of Total Revenue: {Top5RevPerc.round(2)}%")

Top Item share of total purchase count: 1.54%
Top 5 Items share of total purchase count: 6.03%
Top Item share of Total Revenue: 2.13%
Top 5 Items share of Total Revenue: 8.82%


# Report
## Trend 1:
### Female players tend to spend about 10% more per person than male players. Other/Non-disclosed gender players appear to spend even more than female players, but the sample size for these players is too small to be sure of this.

## Trend 2:
### No one particular item drives revenue. Revenue is distributed fairly evenly through most of the items.
### As seen above, Oathbreaker only accounts for 1.54% of all transactions; the top 5 items together only account for 6.03% of all transactions. Likewise, Oathbreaker only accounts for 2.13% of revenue, and the top 5 items only account for 8.82% of all revenue. 